In [1]:
import ipywidgets as widgets
import os, json, torch, pickle
import numpy as np
import vosk
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
import sounddevice as sd
from scipy.io.wavfile import write, read
import ipywidgets as widgets
from IPython.display import display, Audio
from tqdm import tqdm
from pydub.playback import play
from pydub import AudioSegment
import soundfile as sf
import wave
import librosa
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import json
from transformers import Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer
import matplotlib.pyplot as plt
import Levenshtein
from fuzzywuzzy import process
from transformers import BertTokenizer
import torch
from difflib import get_close_matches
from fuzzywuzzy import process
import requests
import inflect
import re
import torchaudio
from torchaudio.transforms import MelSpectrogram
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import random
import torch
import torch.nn as nn
import torch.optim as optim
import tarfile
from sklearn.model_selection import train_test_split
import shutil
import threading


C:\Users\edayo\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Create module for recording audio

In [2]:
# # Define global variables
# duration = 5
# sample_rate = 44100
# recording = None

# def start_recording(button):
#     global recording
#     print("Recording...")

#     recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
#     sd.wait()  # Wait until recording is finished
#     print("Recording finished.")

# def stop_recording(button):
#     # In this simplified example, stop_recording doesn't do anything since sd.wait() in start_recording waits for the duration
#     pass

# def save_audio(button):
#     global recording
#     output_directory = "D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs"
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)
#         num_audio_files = 0
#     else:
#         num_audio_files = len([name for name in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, name))])

#     num_str = f"{num_audio_files+1:04}"  # Adjust 4 as needed for more digits
#     output_filename = f"res_{num_str}.wav"

#     if recording is not None and recording.size > 0:
#         output_file_path = os.path.join(output_directory, output_filename)
#         save_recording(output_file_path, recording)
#         print(f"Audio saved to {output_file_path}")
#     else:
#         print("No recording available to save.")

# def save_recording(filename, data):
#     # Write the recorded data to a WAV file
#     with wave.open(filename, 'w') as wf:
#         wf.setnchannels(2)
#         wf.setsampwidth(2)  # 16-bit audio
#         wf.setframerate(sample_rate)
#         wf.writeframes(data.tobytes())

In [3]:
# Define the global variables
recording = None
recording_event = None
sample_rate = 44100  # Sample rate in Hz

def start_recording(button):
    global recording_event
    duration = 10  # Duration in seconds
    print("Recording...")
    recording_event = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')

def stop_recording(button):
    global recording, recording_event
    sd.stop()
    recording = recording_event.copy()
    print("Recording stopped.")

def play_audio(button):
    if recording is not None:
        display(Audio(data=recording.T, rate=sample_rate))
    else:
        print("No recording available to play.")

def save_audio(button):
    global recording
    output_directory = "D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        num_audio_files = 0
    else:
        num_audio_files = len([name for name in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, name))])

    num_str = f"{num_audio_files+1:04}"  # Adjust 4 as needed for more digits
    output_filename = f"res_{num_str}.wav"

    if recording is not None and recording.size > 0:
        output_file_path = os.path.join(output_directory, output_filename)
        print(f"Recording shape: {recording.shape}, dtype: {recording.dtype}")
        save_recording(output_file_path, recording)
        print(f"Audio saved to {output_file_path}")
    else:
        print("No recording available to save.")

def normalize_audio(data):
    max_val = np.iinfo(np.int16).max
    min_val = np.iinfo(np.int16).min
    max_data = max(abs(data.max()), abs(data.min()))
    if max_data > 0:
        normalized_data = data * (max_val / max_data)
        normalized_data = np.clip(normalized_data, min_val, max_val)
        return normalized_data.astype(np.int16)
    return data

def save_recording(filename, data):
    # Normalize audio data
    data = normalize_audio(data)
    
    # Ensure data is in the correct format
    if data.dtype != np.int16:
        data = data.astype(np.int16)
    
    # Ensure data is 2D
    if data.ndim == 1:
        data = data[:, np.newaxis]
    
    # Write the recorded data to a WAV file
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(data.shape[1])
        wf.setsampwidth(2)  # 16-bit audio
        wf.setframerate(sample_rate)
        wf.writeframes(data.tobytes())

# Create buttons
start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")
play_button = widgets.Button(description="Play Recording")
save_button = widgets.Button(description="Save Recording")

# Assign functions to buttons
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
play_button.on_click(play_audio)
save_button.on_click(save_audio)

# Display buttons
display(start_button)
display(stop_button)
display(play_button)
display(save_button)

Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Button(description='Play Recording', style=ButtonStyle())

Button(description='Save Recording', style=ButtonStyle())

### Denoising Audio (Discontinued)

In [73]:
# def print_directory_structure(root_dir, depth=3):
#     for root, dirs, files in os.walk(root_dir):
#         level = root.replace(root_dir, '').count(os.sep)
#         if level < depth:
#             indent = ' ' * 4 * (level)
#             print(f'{indent}{os.path.basename(root)}/')
#             sub_indent = ' ' * 4 * (level + 1)
#             for f in files:
#                 print(f'{sub_indent}{f}')
#         elif level == depth:
#             print(f'{indent}...')

# # print_directory_structure("D:/Coding Projects/OJT2/train-clean-100/LibriSpeech/")

In [72]:
# def flatten_librispeech_dataset(input_dir, output_dir):
#     os.makedirs(output_dir, exist_ok=True)
    
#     for root, _, files in os.walk(input_dir):
#         for file in files:
#             if file.endswith(".flac"):
#                 src_path = os.path.join(root, file)
#                 dst_path = os.path.join(output_dir, file)
#                 shutil.copy(src_path, dst_path)
#                 print(f"Copied {src_path} to {dst_path}")

# # Example usage
# input_dir = "D:/Coding Projects/OJT2/train-clean-100/LibriSpeech"
# output_dir = "D:/Coding Projects/OJT2/train-clean-100/flattened"
# flatten_librispeech_dataset(input_dir, output_dir)

In [71]:
# fileids = []
# for root, _, files in os.walk("D:/Coding Projects/OJT2/train-clean-100"):
#     for file in files:
#         if file.endswith(".flac"):
#             fileid = os.path.splitext(file)[0]
#             fileids.append(fileid)

# # Load all data
# data = []
# for fileid in fileids:
#     try:
#         waveform, sample_rate, transcript = load_librispeech_item(fileid, "D:/Coding Projects/OJT2/train-clean-100", ".flac", ".txt")
#         data.append((waveform, sample_rate, transcript))
#     except FileNotFoundError as e:
#         print(e)
#     except Exception as e:
#         print(f"Error loading {fileid}: {e}")

# # Split data into training and test sets
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# # Print the size of each set
# print(f"Training set size: {len(train_data)}")
# print(f"Test set size: {len(test_data)}")

In [ ]:
# class NoisyCleanAudioDataset(Dataset):
#     def __init__(self, noisy_samples, clean_samples, transform=None):
#         self.noisy_samples = noisy_samples
#         self.clean_samples = clean_samples
#         self.transform = transform

#     def __len__(self):
#         return len(self.noisy_samples)

#     def __getitem__(self, idx):
#         noisy_waveform = self.noisy_samples[idx]
#         clean_waveform = self.clean_samples[idx]
        
#         if self.transform:
#             noisy_waveform = self.transform(noisy_waveform)
#             clean_waveform = self.transform(clean_waveform)
        
#         return noisy_waveform, clean_waveform

# # Create dataset and dataloader
# dataset = NoisyCleanAudioDataset(noisy_samples, clean_samples, transform=MelSpectrogram())
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# class ConvAutoencoder(nn.Module):
#     def __init__(self):
#         super(ConvAutoencoder, self).__init__()
        
#         # Encoder
#         self.encoder = nn.Sequential(
#             nn.Conv1d(1, 16, kernel_size=3, stride=2, padding=1),
#             nn.ReLU(),
#             nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
#             nn.ReLU(),
#             nn.Conv1d(32, 64, kernel_size=3, stride=2, padding=1),
#             nn.ReLU()
#         )
        
#         # Decoder
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose1d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
#             nn.ReLU(),
#             nn.ConvTranspose1d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
#             nn.ReLU(),
#             nn.ConvTranspose1d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x

# model = ConvAutoencoder()

# def train_autoencoder(model, dataloader, num_epochs=20, learning_rate=1e-3):
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
#     for epoch in range(num_epochs):
#         for noisy, clean in dataloader:
#             noisy, clean = noisy.unsqueeze(1), clean.unsqueeze(1)  # Add channel dimension
#             optimizer.zero_grad()
#             outputs = model(noisy)
#             loss = criterion(outputs, clean)
#             loss.backward()
#             optimizer.step()
        
#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# train_autoencoder(model, dataloader)

In [ ]:
# def denoise_audio(model, noisy_waveform):
#     model.eval()
#     with torch.no_grad():
#         noisy_waveform = noisy_waveform.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
#         denoised_waveform = model(noisy_waveform)
#         denoised_waveform = denoised_waveform.squeeze(0).squeeze(0)  # Remove batch and channel dimensions
#     return denoised_waveform

# some_noisy_waveform, _ = torchaudio.load("path/to/new/noisy/file.wav")
# denoised_waveform = denoise_audio(model, some_noisy_waveform)

# torchaudio.save("path/to/save/denoised_file.wav", denoised_waveform, 16000)

### Record your audio

### Test your audio

In [4]:
audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs'
num_audio_files = len([name for name in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, name))])

audio_files = []
for i in range(num_audio_files):
    num_str = f"{i+1:04}"
    audio_path = os.path.join(audio_dir, f'res_{num_str}.wav')
    audio_files.append(audio_path)
    
audio_files

['D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0001.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0002.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0003.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0004.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0005.wav']

In [5]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [6]:
test_transcript = []
def show_transcription(model, processor, audio_file):
    audio_input, sampling_rate = librosa.load(audio_file, sr=16000)
    inputs = processor(audio_input, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription

In [ ]:
for audio_file in audio_files:
    out = show_transcription(model, processor, audio_file)
    test_transcript.append(out)

## Prepare dataset for fine-tuning (discontinued)

In [9]:
def getFileNames(audio_dir, isWav):
    num_files = len([name for name in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, name))])

    files = []
    for i in range(num_files):
        num_str = f"{i+1:04}"
        if isWav:
            path = os.path.join(audio_dir, f'res_{num_str}.wav')
        else:
            path = os.path.join(audio_dir, f'res_{num_str}.txt')

        files.append(path)

    return files

audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_audio_female'
listFemaleAudio = getFileNames(audio_dir, True)
audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_audio_child'
listChildAudio = getFileNames(audio_dir, True)
audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_audio_male'
listMaleAudio = getFileNames(audio_dir, True)
audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_text'
listAudioLabels = getFileNames(audio_dir, False)

In [13]:
class AudioDataset(Dataset):
    def __init__(self, audio_files, label_files, processor):
        self.audio_files = audio_files
        self.label_files = label_files
        self.processor = processor

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        label_file = self.label_files[idx]

        # Load audio file
        audio_input, sampling_rate = librosa.load(audio_file, sr=16000)
        
        # Load transcription
        with open(label_file, 'r', encoding='utf-8') as f:
            transcription = f.readline().strip()

        # Process audio input and transcription
        inputs = self.processor(audio_input, sampling_rate=sampling_rate, return_tensors="pt")
        with self.processor.as_target_processor():
            labels = self.processor(transcription, return_tensors="pt").input_ids

        return {
            'input_values': inputs.input_values[0],  # Flatten the input tensor
            'labels': labels[0]  # Flatten the label tensor
        }
    
def collate_fn(batch):
    input_values = [item['input_values'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Ensure input_values is a dictionary with the correct key for padding
    processed_inputs = {"input_values": input_values}

    # Pad inputs
    input_values_padded = processor.feature_extractor.pad(processed_inputs, padding=True, return_tensors="pt").input_values

    # Pad labels
    with processor.as_target_processor():
        labels_padded = processor.pad({"input_ids": labels}, padding=True, return_tensors="pt").input_ids

    return {
        'input_values': input_values_padded,
        'labels': labels_padded
    }

In [10]:
all_audio_files = listFemaleAudio + listChildAudio + listMaleAudio
all_label_files = listAudioLabels + listAudioLabels + listAudioLabels

### Training model to ptrdict the transcript (no text pre-processing)

In [12]:
file_contents = []
for file_path in listAudioLabels:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        file_contents.append(content)

In [13]:
def get_sublist_with_numbers(strings):
    # Regular expression to check for the presence of digits in a string
    pattern = r'\d'
    # Filter the list to include only strings that contain a number
    sublist_with_numbers = [s for s in strings if re.search(pattern, s)]
    return sublist_with_numbers

In [14]:
sublist_with_numbers = get_sublist_with_numbers(file_contents)

In [19]:
# print(sublist_with_numbers)

In [15]:
dictionaryTagalogRanges = {
    "eleven minutes" : "11 hanggang 20 minuto",
    "tweleve minutes" : "11 hanggang 20 minuto",
    "thirteen minutes" : "11 hanggang 20 minuto",
    "fourteen minutes" : "11 hanggang 20 minuto",
    "fifteen minutes" : "11 hanggang 20 minuto",
    "sixteen minutes" : "11 hanggang 20 minuto",
    "seventeen minutes" : "11 hanggang 20 minuto",
    "eighteen minutes" : "11 hanggang 20 minuto",
    "nineteen minutes" : "11 hanggang 20 minuto",
    "twenty minutes" : "11 hanggang 20 minuto",
#     "four" : '4 to 7',
#     "five" : '4 to 7',
#     "six" : '4 to 7',
#     "seven": '4 to 7',
    "isang araw nakalipas": '2 - 4 araw nakalipas',
    "dalawang araw nakalipas": '2 - 4 araw nakalipas', 
    "tatlong araw nakalipas": '2 - 4 araw nakalipas', 
    "apat araw nakalipas": '2 - 4 araw nakalipas',
    "tatlong beses": '3  - 5 beses', 
    "apat na beses": '3  - 5 beses', 
    "limang beses": '3  - 5 beses', 
    "anim na beses": '4 - 6 beses', 
    "dalawang araw lumipas": '2 - 4 araw lumipas', 
    "tatlong araw lumipas": '2 - 4 araw lumipas', 
    "apat na araw lumipas": '2 - 4 araw lumipas', 
    "isang araw": '1 - 3 araw', 
    "dalawang araw": '1 - 3 araw', 
    "tatlong araw": '1 - 3 araw', 
    "pito na araw": '7 - 10 na araw', 
    "walo na araw": '8 - 10 na araw', 
    "siyam na araw": '8 - 10 na araw', 
    "sampu na araw": '8 - 10 na araw', 

    "limang araw lumipas": '5 - 7 araw lumipas', 
    "anim araw lumipas": '5 - 7 araw lumipas', 
    "pitong araw lumipas": '5 - 7 araw lumipas', 

    "apat na araw": '4-6 araw', 
    "lima na araw": '4-6 araw', 
    "anim na araw": '4-6 araw', 

#     '1 - 3', 
    "lima na araw lumipas": '5 - 7 na araw lumipas', 
    "anim na araw lumipas": '5 - 7 na araw lumipas', 
    "pito na araw lumipas": '5 - 7 na araw lumipas', 

    "lima na araw": '5 - 7 na araw', 
    "anim na araw": '5 - 7 na araw', 
    "isang linggo": '1 - 2 linggo', 
    "dalawang linggo": '1 - 2 linggo', 

    "limang araw nakalipas": '5 - 7 araw nakalipas', 
    "anim araw nakalipas": '5 - 7 araw nakalipas', 
    "pitong araw nakalipas": '5 - 7 araw nakalipas', 

    "isang minuto": '1 - 5 minuto',
    "dalawang minuto": '1 - 5 minuto', 
    "tatlong minuto": '1 - 5 minuto', 
    "apat na minuto": '1 - 5 minuto', 
    "limang minuto": '1 - 5 minuto', 

    "limang araw": '5 araw - 1 linggo',
    "anim na araw": '5 araw - 1 linggo', 
    "issang linggo": '5 araw - 1 linggo', 

    "limang araw": '5 araw -1 linggo',
    "anim na araw": '5 araw -1 linggo', 
    "issang linggo": '5 araw -1 linggo', 
    
    "dalawang araw":'2 - 3 araw', 
    "tatlong araw":'2 - 3 araw', 

    "dalawang beses": '2 - 3 beses', 
    "tatlong beses": '2 - 3 beses', 

    "isang araw": '1 - 3 na araw', 
    "dalawang araw": '1 - 3 na araw', 
    "tatlong araw": '1 - 3 na araw', 

    "isang beses": '1 - 3 beses', 
    "dalawang beses": '1 - 3 beses', 
    "tatlong beses": '1 - 3 beses', 

#     '4 - 6', 
    "isang oras": '1 - 5 na oras', 
    "dalawang oras": '1 - 5 na oras', 
    "tatlong oras": '1 - 5 na oras', 
    "apat na oras": '1 - 5 na oras', 
    "limang oras": '1 - 5 na oras', 

    "isang minuto" :'1 - 30 na minuto',
    "five minutes" :'1 - 30 na minuto', 
    "sampung minuto" :'1 - 30 na minuto', 
    "fifteen minutes" :'1 - 30 na minuto', 
    "twenty minutes" :'1 - 30 na minuto', 
    "twenty five minutes" :'1 - 30 na minuto',
    "twenty five minutes" :'1 - 30 na minuto', 



    "apat na araw": '4 - 6 na araw',
    "lima na araw": '4 - 6 na araw',
    "anim na araw": '4 - 6 na araw',

    "dalawang araw lumipas": '2 - 4 na araw lumipas', 
    "tatlong araw lumipas": '2 - 4 na araw lumipas', 
    "apat na araw lumipas": '2 - 4 na araw lumipas', 

    "isang araw": '1-3 araw', 
    "dalawang araw": '1-3 araw', 
    "tatlong araw": '1-3 araw', 

    "apat na araw": '2 - 4 na araw', 
    
    "thirty minutes": '31 - 60 na minuto', 
    "fourty minutes": '31 - 60 na minuto', 
    "fifty minutes": '31 - 60 na minuto',
    "sixty minutes": '31 - 60 na minuto', 
    "one hour": '31 - 60 na minuto', 

    "six minutes": '6 - 10 minuto',
    "seven minutes": '6 - 10 minuto',
    "eight minutes": '6 - 10 minuto',
    "nine minutes": '6 - 10 minuto',
    "ten minutes": '6 - 10 minuto',
}

In [16]:
keys_list = list(dictionaryTagalogRanges.keys())

In [17]:
file_contents

['higit sa 7',
 'wala pang 3 beses',
 '2 - 4 araw nakalipas',
 'pagkaing dagat',
 'dalawa',
 'pangangati',
 'malambot',
 '3  - 5 beses',
 'paggamot ng root canal',
 'usok/alikabok',
 'brownish',
 'pasta/spaghetti',
 '4 - 6 beses',
 'wala pang 1 linggo',
 '2 - 4 araw lumipas',
 '38.3 - 38.6',
 'dalawa kada linggo',
 'near-sighted',
 'pre existing condition',
 'itim',
 'loob',
 'ubo',
 'wala pang 1 araw',
 'isa kada linggo',
 '38',
 'dalawa kada araw',
 'medication',
 '1 - 3 araw',
 'aso',
 'pusa',
 '11 hanggang 20 minuto',
 '3',
 '7 - 10 na araw',
 '0',
 'medyo',
 'higit 1 linggong nakalipas',
 'tuyo',
 'neozep',
 '8 - 10 na araw',
 'foreign object',
 'dahil sa mga fluids',
 'higit sa 10 minuto',
 'avulsion',
 'asul',
 'puncture',
 'exposure sa chlorine',
 'kanina/ngayon lang',
 '5 - 7 araw lumipas',
 'tulog',
 'lima',
 'nadulas/nahulog',
 'tinapay/pandesal',
 'pula',
 '9',
 '7 pataas',
 '4-6 araw',
 'may dugo',
 '1 - 3',
 'pisikal na ehersisyo',
 'wala pa isang linggo',
 'higit sa 5 be

In [18]:
p = inflect.engine()

def number_to_text(number):
    return p.number_to_words(number)

def range_to_text(range_str):
    # Check if the range contains a decimal point
    if '.' in range_str:
        # Split the range string by hyphen
        start, end = range_str.split('-')
        start = start.strip()
        end = end.strip()

        # Convert both parts of the range to text
        start_text = number_to_text(start)
        end_text = number_to_text(end)

        return f"{start_text} to {end_text}"
    else:
        return range_str  # Return the original string if no decimal point


extract values with hyphens (which tells about ranges)

In [24]:
# hyphenated_number_strings = [s for s in file_contents if '-' in s and re.search(r'\d', s)]
# print(hyphenated_strings)

Convert values with decimal places to text

In [25]:
# text_only_strings = [range_to_text(s) for s in hyphenated_number_strings]
# print(text_only_strings)

In [19]:
def process_list(file_contents):
    for i, s in enumerate(file_contents):
        if '-' in s and re.search(r'\d', s):
            file_contents[i] = range_to_text(s)
    return file_contents

# Apply the function to the list
processed_file_contents = process_list(file_contents)

print(processed_file_contents)

['higit sa 7', 'wala pang 3 beses', '2 - 4 araw nakalipas', 'pagkaing dagat', 'dalawa', 'pangangati', 'malambot', '3  - 5 beses', 'paggamot ng root canal', 'usok/alikabok', 'brownish', 'pasta/spaghetti', '4 - 6 beses', 'wala pang 1 linggo', '2 - 4 araw lumipas', 'thirty-eight point three to thirty-eight point six', 'dalawa kada linggo', 'near-sighted', 'pre existing condition', 'itim', 'loob', 'ubo', 'wala pang 1 araw', 'isa kada linggo', '38', 'dalawa kada araw', 'medication', '1 - 3 araw', 'aso', 'pusa', '11 hanggang 20 minuto', '3', '7 - 10 na araw', '0', 'medyo', 'higit 1 linggong nakalipas', 'tuyo', 'neozep', '8 - 10 na araw', 'foreign object', 'dahil sa mga fluids', 'higit sa 10 minuto', 'avulsion', 'asul', 'puncture', 'exposure sa chlorine', 'kanina/ngayon lang', '5 - 7 araw lumipas', 'tulog', 'lima', 'nadulas/nahulog', 'tinapay/pandesal', 'pula', '9', '7 pataas', '4-6 araw', 'may dugo', '1 - 3', 'pisikal na ehersisyo', 'wala pa isang linggo', 'higit sa 5 beses', 'hindi kailanma

In [70]:
numericals = {
"31 - 60":"thirty one to sixty", 
# "31 - 60":"tatlumput isa hanggang animnapu",
# "two to four", 
"2 - 4": "dalawa hanggang apat",
# "one to three", 
# "four to six", 
"4 - 6": "apat hanggang anim",
# "one to thirty minutes", 
"1 - 30":"isa hanggang tatlumpung minuto",
"1 - 3": "isa hanggang tatlo",

# "one to five", 
"1 - 5":"isa hanggang lima",
# "thirty seven point five to thirty seven point eight", 
# "37.5 - 37.8":"tatlumput pito point lima hanggang tatlumpu't pito point walo",
# "one to ten", 
"1 - 10":"isa hanggang sampu",
# "thirty seven point nine to thirty eight point two",
# "37.9 - 38.2":"tatlumpu't pito point siyam hanggang tatlumput walo point dalawa",
# "thirty eight point three to thirty eight point six", 
# "38.3 - 38.6":"tatlumput walo point tatlo hanggang tatlumput walo point anim",
# "thirty eight point seven",  
# "38.7": "tatlumput walo point pito",
# "twenty", 
"20": "dalawampu",
"11 - 20":"eleven to twenty", 
# "11 - 20":"labing isa hanggang dalawampu",
# "thirty nine", 
"39": "tatlumput siyam",
"30": "tatlumput",
# "forty", 
"40":"apatnapu",
# "twenty-four", 
"24":"dalawamput apat",
# "thirty-nine", 
"38": "tatlumput siyam",
# "six", 
"6": "anim",
# "ten", 
"10": "sampu",
# "seven", 
"7":"pito",
# "five", 
"5":"lima",
# "one", 
"11":"labing-isa",
"1":"isa",
# "eight", 
"9": "syam",
"8":"walo",
"3": "tatlo",
"4": "apat",
"0": "zero",
"2": "dalawa",
# "-": " hanggang "
}

In [62]:
def replace_numbers(text, numericals):
    for number, word in numericals.items():
        text = text.replace(number, word)
    return text

def replace_hyphen(text):
    # Regular expression pattern to match hyphens enclosed by numbers or space + numbers
    pattern = r'(?<=\d|\d )-(?=\d|\d )'    
    replaced_text = re.sub(pattern, ' hanggang ', text)
    return replaced_text

In [63]:
# tmp_contents = [replace_numbers(sentence, numericals) for sentence in file_contents]
updated_contents = [replace_numbers(sentence, numericals) for sentence in processed_file_contents]

# Print updated contents
for sentence in updated_contents:
    print(sentence)

higit sa pito
wala pang tatlo beses
dalawa hanggang apat araw nakalipas
pagkaing dagat
dalawa
pangangati
malambot
tatlo  - lima beses
paggamot ng root canal
usok/alikabok
brownish
pasta/spaghetti
apat hanggang anim beses
wala pang isa linggo
dalawa hanggang apat araw lumipas
thirty-eight point three to thirty-eight point six
dalawa kada linggo
near-sighted
pre existing condition
itim
loob
ubo
wala pang isa araw
isa kada linggo
tatlumput siyam
dalawa kada araw
medication
isa hanggang tatlo araw
aso
pusa
labing-isa hanggang dalawampu minuto
tatlo
pito - sampu na araw
zero
medyo
higit isa linggong nakalipas
tuyo
neozep
walo - sampu na araw
foreign object
dahil sa mga fluids
higit sa sampu minuto
avulsion
asul
puncture
exposure sa chlorine
kanina/ngayon lang
lima - pito araw lumipas
tulog
lima
nadulas/nahulog
tinapay/pandesal
pula
syam
pito pataas
apat-anim araw
may dugo
isa hanggang tatlo
pisikal na ehersisyo
wala pa isang linggo
higit sa lima beses
hindi kailanman
buwan
lima - pito na ar

In [23]:
for sentence in updated_contents:
    if "hanggang" in sentence:
        print(sentence)

dalawa hanggang apat araw nakalipas
apat hanggang anim beses
dalawa hanggang apat araw lumipas
isa hanggang tatlo araw
labing-isa hanggang dalawampu minuto
isa hanggang tatlo
isa hanggang lima minuto
isa hanggang sampu minuto
isa hanggang tatlo na araw
isa hanggang tatlo beses
apat hanggang anim
isa hanggang lima na oras
isa hanggang tatlozero na minuto
apat hanggang anim na araw
dalawa hanggang apat na araw lumipas
dalawa hanggang apat na araw


In [64]:
new_file_contents  = updated_contents + keys_list

In [59]:
new_file_contents

['higit sa pito',
 'wala pang tatlo beses',
 'dalawa hanggang apat araw nakalipas',
 'pagkaing dagat',
 'dalawa',
 'pangangati',
 'malambot',
 'tatlo  - lima beses',
 'paggamot ng root canal',
 'usok/alikabok',
 'brownish',
 'pasta/spaghetti',
 'apat hanggang anim beses',
 'wala pang isa linggo',
 'dalawa hanggang apat araw lumipas',
 'thirty-eight point three to thirty-eight point six',
 'dalawa kada linggo',
 'near-sighted',
 'pre existing condition',
 'itim',
 'loob',
 'ubo',
 'wala pang isa araw',
 'isa kada linggo',
 'tatlumput siyam',
 'dalawa kada araw',
 'medication',
 'isa hanggang tatlo araw',
 'aso',
 'pusa',
 'labing-isa hanggang dalawampu minuto',
 'tatlo',
 'pito - sampu na araw',
 'zero',
 'medyo',
 'higit isa linggong nakalipas',
 'tuyo',
 'neozep',
 'walo - sampu na araw',
 'foreign object',
 'dahil sa mga fluids',
 'higit sa sampu minuto',
 'avulsion',
 'asul',
 'puncture',
 'exposure sa chlorine',
 'kanina/ngayon lang',
 'lima - pito araw lumipas',
 'tulog',
 'lima',

### Create data for training

In [41]:
display(start_button)
display(stop_button)
display(play_button)
display(save_button)

Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Button(description='Play Recording', style=ButtonStyle())

Button(description='Save Recording', style=ButtonStyle())

Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0001.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0002.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0003.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0004.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0005.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dtype: int16
Audio saved to D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0006.wav
Recording...
Recording stopped.
Recording shape: (441000, 2), dt

In [39]:
random_element = random.sample(new_file_contents, 2)
print(random_element)

['kulay rosas', 'nadulas/nahulog']


In [42]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
transcript = []
target = []
def show_transcription(model, processor, audio_file):
    audio_input, sampling_rate = librosa.load(audio_file, sr=16000)
    inputs = processor(audio_input, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription
    
# for audio_file, label_file in zip(all_audio_files, all_label_files):
#     output = show_transcription(model, processor, audio_file)
#     transcript.append(output)
    
#     with open(label_file, 'r', encoding='utf-8') as file:
#         content = file.read()
#         target.append(content)

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [43]:
updated_contents[:10]

['higit sa pito',
 'wala pang tatlo beses',
 'dalawa hanggang apat araw nakalipas',
 'pagkaing dagat',
 'dalawa',
 'pangangati',
 'malambot',
 'tatlo  - lima beses',
 'paggamot ng root canal',
 'usok/alikabok']

In [30]:
# for item1, item2 in zip(transcript, updated_contents):
#     print(item1, item2)

### Use the samples

In [44]:
def map_transcripts(transcripts, labels):
    mapped_results = []
    for trans in transcripts:
        trans_str = ' '.join(trans)
        trans_length = len(trans_str)
        
        # Filter labels based on length difference
        filtered_labels = [label for label in labels if abs(len(label) - trans_length) <= 5]
        
        if filtered_labels:
            match, score = process.extractOne(trans_str, filtered_labels)
            result = process.extract(trans_str, filtered_labels)
            print(result)
            mapped_results.append((trans_str, match, score))
        else:
            mapped_results.append((trans_str, None, 0))
            
    return mapped_results

# mapped_transcripts = map_transcripts(transcript, updated_contents)

# Calculate accuracy and print results
def print_result(mapped_transcripts):
    for original, mapped, score in mapped_transcripts:
        print(f"Original: {original} -> Mapped: {mapped} (Score: {score})")

    # Calculate accuracy
#     correct_matches = sum(1 for _, mapped, _ in mapped_transcripts if mapped)
#     accuracy = correct_matches / len(mapped_transcripts) * 100
#     print(f"Accuracy: {accuracy:.2f}%")

# print_result(mapped_transcripts)

### Test my speech transcription

In [61]:
# test_transcript

[]

In [59]:
# mapped_transcripts = map_transcripts(test_transcript, updated_contents)
# print_result(mapped_transcripts)

### Test API

In [66]:
audio_dir = 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs'
num_audio_files = len([name for name in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, name))])

audio_files = []
for i in range(num_audio_files):
    num_str = f"{i+1:04}"
    audio_path = os.path.join(audio_dir, f'res_{num_str}.wav')
    audio_files.append(audio_path)
    
audio_files

['D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0001.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0002.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0003.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0004.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0005.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0006.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0007.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0008.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0009.wav',
 'D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\\res_0010.wav']

In [67]:
transcript = []
for audio_file in zip(audio_files):
    print(audio_file[0])
    output = show_transcription(model, processor, audio_file[0])
    transcript.append(output)

D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0001.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0002.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0003.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0004.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0005.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0006.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0007.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0008.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0009.wav
D:/Coding Projects/OJT2/tagalog/dataset_audio/inputs\res_0010.wav


In [49]:
print(transcript)

[['ABUTNAARO'], ['THE LAWAN ARAUDOMIPAS'], ['HALLO WHERE IS THAT LONG ARRO'], ['FIFTY MINUTES'], ['ONE AR']]


In [68]:
mapped_transcripts = map_transcripts(transcript, new_file_contents)
print_result(mapped_transcripts)

[('lima na araw', 78), ('limang araw', 73), ('limang oras', 64), ('nalimutan ko', 61), ('anim na araw', 61)]
[('twenty five minutes', 100), ('twenty minutes', 95), ('tweleve minutes', 76), ('fifteen minutes', 72), ('nineteen minutes', 70)]
[('isa hanggang tatlozero na minuto', 68), ('labing-isa hanggang dalawampu minuto', 58), ('dalawa hanggang apat na araw', 49), ('higit isa linggong nakalipas', 47), ('dalawa hanggang apat na araw lumipas', 46)]
[('isang linggo', 73), ('issang linggo', 70), ('isang minuto', 64), ('linggo', 60), ('isa kada linggo', 56)]
[('sixteen minutes', 100), ('fifteen minutes', 87), ('sixty minutes', 86), ('seven minutes', 86), ('six minutes', 85)]
[('antibiotics', 100), ('anti histamine', 56), ('matubig', 51), ('gamot isa', 50), ('maitim', 47)]
[('apat hanggang anim na araw', 88), ('isa hanggang lima na oras', 73), ('isa hanggang tatlo na araw', 72), ('apat hanggang anim beses', 71), ('dalawa hanggang apat na araw', 69)]
[('lima - pito na araw', 73), ('lima - pit

In [49]:
max_cap = len(file_contents)

In [69]:
for item in mapped_transcripts:
    print(item)
    if item[1] in new_file_contents[max_cap:]:
        print("Corresponding to this choice range: ", dictionaryTagalogRanges[item[1]])

('LIMANA ARRO', 'lima na araw', 78)
Corresponding to this choice range:  4 - 6 na araw
('TWENTY FIVE MINUTES', 'twenty five minutes', 100)
Corresponding to this choice range:  1 - 30 na minuto
('ISA HANGARD THAT LONG PUT MINU DO', 'isa hanggang tatlozero na minuto', 68)
('ISANLINGUM', 'isang linggo', 73)
Corresponding to this choice range:  1 - 2 linggo
('SIXTEEN MINUTES', 'sixteen minutes', 100)
Corresponding to this choice range:  11 hanggang 20 minuto
('ANTIBIOTICS', 'antibiotics', 100)
('APAT HANGANG ALIM NA ARO', 'apat hanggang anim na araw', 88)
('LIMA HAMDAM  PITO NAADO', 'lima - pito na araw', 73)
('CULAY ROSAS', 'kulay rosas', 91)
('NADULAS OR NAHOLU', 'nadulas/nahulog', 77)


In [ ]:
# Define the API endpoint
url = "http://127.0.0.1:8000/transcribe/"

# Path to the audio file you want to upload
audio_file_path = audio_files[2]

# Open the audio file in binary mode
with open(audio_file_path, "rb") as audio_file:
    # Send a POST request to the API endpoint with the audio file
    files = {"file": audio_file}
    response = requests.post(url, files=files)

# Print the response from the server
if response.status_code == 200:
    print("Result-->", response.json())
else:
    print("Failed to transcribe audio. Status code:", response.status_code)

In [ ]:
response.json()["transcription"][1]

Use dictionary to map the transcription to close range

response ==> dictionaryTagalogRanges ==> new_file_contents ==> new_file_content

In [ ]:
random_element = random.sample(new_file_contents, 5)
print(random_element)

In [ ]:
for i, j in zip(file_contents, new_file_contents):
    print(f"{i} @ {j}")